# Connecting to GCS Using Local Spark

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import types

In [2]:
creds_path = "/home/jose/Documents/projects/de-zoomcamp-fa/creds/de-zc-i-e4c405955ac3.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName("test") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", creds_path)

#### Setting the Spark Context
What is the code below doing?
- the `fs.` in the `hadoop_conf` variable is saying, "when you see the file system that starts with 'gs' (GCS), you need to use the google hadoop file system implementation coming from the google cloud connector .jar file"
- AND "you need to use the service account credentials (third `hadoop_conf` configuration"

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", creds_path)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/06/22 16:35:05 WARN Utils: Your hostname, jose-MacBookPro resolves to a loopback address: 127.0.1.1; using 192.168.1.224 instead (on interface wlp2s0)
23/06/22 16:35:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/22 16:35:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Now, we can create the session:

In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
path = "gs://dtc_data_lake_de-zc-i/pq/green/*/*"
df_grn = spark.read.parquet(path)

In [7]:
df_grn.count()

2304517